# Nth order Taylor method

$$
y_{i+1} = y_i + hT^{(N)}_i \Longrightarrow T^{(N)}_i = \sum_{n=0}^N \frac{h^n}{(n+1)!} f^{(n)}(t_i, y_i)
$$

In [4]:
import sympy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool
import time

In [30]:
from math import factorial

class Nth_Taylor:
    def __init__(self, h: float, t0: float, y0: float, fs: tuple|list, executor=None):
        self.h = h
        self.y = [y0]
        self.t = [t0]
        self.fs = fs # [f, f', f'', f''', ...]
        self.N = tuple(range(len(fs))) # [0, 1, 2, ...]
        self.i = 0
        self.executor = executor

    def _T(self, n):
        return self.h**n * self.fs[n](self.t[-1], self.y[-1])/factorial(n + 1)
    
    def step(self):
        T = sum(self.executor.map(self._T, self.N) if self.executor else map(self._T, self.N))
        self.y.append(
            self.y[-1] + self.h*T
        )
        self.t.append(self.t[-1] + self.h)
        self.i += 1
    
    def solve(self, m: int):
        for _ in range(m):
            self.step()
        return self.t, self.y
    
    def reset(self):
        self.y = [self.y[0]]
        self.t = [self.t[0]]

In [68]:
def f(t, y):
    return 2*(1 - t*y)/(t**2 + 1)

t = sp.symbols('t')
y = sp.Function('y')(t)
dy = y.diff(t)
fs = [f]
for _ in range(4):
    fn = fs[-1](t, y).diff(t).subs(dy, f(t, y)).simplify()
    fs.append(sp.lambdify((t, y), fn))

exemple = Nth_Taylor(
    h= 0.2,
    t0= 0,
    y0= 0.5,
    fs= fs,
)

In [69]:
%%time
exemple.reset()
exemple.solve(1000000)
None

CPU times: user 4.23 s, sys: 23.9 ms, total: 4.26 s
Wall time: 4.25 s


In [70]:
%%time
exemple.reset()
with ThreadPoolExecutor(max_workers=len(exemple.fs)) as executor:
    exemple.solve(1000000)

CPU times: user 4.21 s, sys: 31.9 ms, total: 4.25 s
Wall time: 4.24 s


In [71]:
%%time
exemple.reset()
with Pool(processes=len(exemple.fs)) as executor:
    exemple.solve(1000000)

CPU times: user 4.2 s, sys: 88 ms, total: 4.29 s
Wall time: 4.31 s


In [18]:
pd.DataFrame(dict(zip(('x', 'y'), solver.solve(10))))

,x,y
0,0.0,0.500000
1,0.2,0.830000
2,0.4,1.215800
3,0.6,1.652076
4,0.8,2.132333
5,1.0,2.648646
6,1.2,3.191348
7,1.4,3.748645
8,1.6,4.306146
9,1.8,4.846299


In [100]:
a = np.array([
    [1, 4, 52, 57],
    [22, 110, -3, 129],
    [22, 2, 14, 38]
]).astype('float16')
for i in range(a.shape[0]-1):
    for j in range(i+1, a.shape[0]):
        print(i, j)
        print(a)
        a[j] -= a[i]*a[j, i]/a[i, i]
a

0 1
[[  1.   4.  52.  57.]
 [ 22. 110.  -3. 129.]
 [ 22.   2.  14.  38.]]
0 2
[[ 1.000e+00  4.000e+00  5.200e+01  5.700e+01]
 [ 0.000e+00  2.200e+01 -1.147e+03 -1.125e+03]
 [ 2.200e+01  2.000e+00  1.400e+01  3.800e+01]]
1 2
[[ 1.000e+00  4.000e+00  5.200e+01  5.700e+01]
 [ 0.000e+00  2.200e+01 -1.147e+03 -1.125e+03]
 [ 0.000e+00 -8.600e+01 -1.130e+03 -1.216e+03]]


/tmp/ipykernel_110644/273545726.py:10: RuntimeWarning: overflow encountered in multiply
  a[j] -= a[i]*a[j, i]/a[i, i]


array([[ 1.000e+00,  4.000e+00,  5.200e+01,  5.700e+01],
       [ 0.000e+00,  2.200e+01, -1.147e+03, -1.125e+03],
       [ 0.000e+00,  0.000e+00,       -inf,       -inf]], dtype=float16)